In [1]:
import os
from itertools import product

import numpy as np

from image.segmentation import BasicImageSegmenter
from ml.eval import eval_model

print(os.getcwd())
os.chdir('/home/jelinek/recetox/')

from ml.pipeline import FeitDataPipeline
from ml.util import NeighborhoodImageDataGenerator, rename_keras_layer

from tensorflow import keras

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.callbacks import ReduceLROnPlateau

from cfg import *

from livelossplot import PlotLossesKerasTF

name = "MySimpleCNN_Inception_module-v2-reduced-neighborhood-add"


/home/jelinek/recetox/src/notebooks


In [2]:
class MySimpleCNNInceptionModule(FeitDataPipeline):

    def __init__(self, neighborhood_tiles: int, *args, **kwargs):

        self.neighborhood_tiles: int = neighborhood_tiles

        super().__init__(*args, **kwargs)

        self.model = self.get_compiled_model()
        self.params.name = name
        self.params.epochs = 9
        self.batch_size = 16
        self.params.tile_size=256
        self.params.number_of_classes=12


    '''
    def get_optimizer(self):
        lr_schedule = schedules.ExponentialDecay(
            initial_learning_rate=1e-2,
            decay_steps=TRAINING_EXAMPLES_COUNT // self.params.batch_size * 30,
            decay_rate=0.1,
            staircase=True)

        return Adam(
            # learning_rate=0.1,
            learning_rate=lr_schedule,
            beta_1=0.99,
            beta_2=0.9999)
    '''


    @staticmethod
    def __get_basic_layers(model_idx: int):
        trained_base_model = keras.models.load_model('saved-models/frozen/MySimpleCnn_Feit-inceptionv2-reduced.h5')
        trained_base_model.trainable = False

        name_suffix = '_model_tile_' + str(model_idx)

        for i in range(len(trained_base_model.layers)):
            rename_keras_layer(trained_base_model, trained_base_model.layers[i],
                               trained_base_model.layers[i].name + name_suffix)

        last_layer = trained_base_model(trained_base_model.inputs, training=False)

        rename_keras_layer(trained_base_model, last_layer, last_layer.name + name_suffix)

        trained_base_model._name += name_suffix
        return trained_base_model.inputs[0], last_layer


    @staticmethod
    def get_compiled_model():
        neighborhood = 1
        neighborhood_size = 2 * neighborhood + 1


        neighborhood_networks = [MySimpleCNNInceptionModule.__get_basic_layers(model_idx)
                                 for model_idx in range(neighborhood_size ** 2)]

        outputs = [model[1] for model in neighborhood_networks]

        #outputs_conv = [keras.layers.Conv1D(filters=1, kernel_size=1)(output) for output in outputs]

        #merge_layer = keras.layers.AdditiveAttention()(outputs)
        merge_layer = keras.layers.Add()(outputs)

        output = keras.layers.Dense(units = 12, activation = 'softmax')(merge_layer)

        inputs = [model[0] for model in neighborhood_networks]
        model = keras.Model(inputs=inputs, outputs=output)
        keras.utils.plot_model(model, to_file='neighborhood_ensemble.png', show_shapes=True)
        model.summary()

        return model


    def _train_model(self, data_train, data_valid):

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                      patience=30, min_lr=1e-4, verbose=1,
                                      cooldown=20)

        self.model.fit(data_train,
                       steps_per_epoch=250,
                       epochs=1,
                       shuffle=True,
                       validation_data=data_valid,
                       validation_freq=5,
                       verbose=1,
                       validation_steps=1860 // 16,
                       callbacks=[self.tensorboard, reduce_lr, PlotLossesKerasTF()])

    def get_data_loader_training(self):
        datagen_train = NeighborhoodImageDataGenerator(self.neighborhood_tiles, horizontal_flip=True, vertical_flip=True, samplewise_center=True,
                                           samplewise_std_normalization=True)
        tiles_per_axis = self.neighborhood_tiles * 2 + 1

        return datagen_train.flow_from_directory(directory=self.params.data_training, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=True,
                                                 target_size=(self.params.tile_size * tiles_per_axis,
                                                              self.params.tile_size * tiles_per_axis))

    def get_data_loader_validation(self):
        datagen_valid = NeighborhoodImageDataGenerator(self.neighborhood_tiles, samplewise_center=True, samplewise_std_normalization=True)
        tiles_per_axis = self.neighborhood_tiles * 2 + 1

        return datagen_valid.flow_from_directory(directory=self.params.data_validation, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=False,
                                                 target_size=(self.params.tile_size * tiles_per_axis,
                                                              self.params.tile_size * tiles_per_axis))

    def get_datagen_segmentation(self):
        return NeighborhoodImageDataGenerator(self.neighborhood_tiles, samplewise_center=True,
                                              samplewise_std_normalization=True)

    def build_segmenter(self):
        tiles_per_axis = self.neighborhood_tiles * 2 + 1

        return BasicImageSegmenter(self.model, self.get_datagen_segmentation(),
                                   tile_size=self.params.tile_size * tiles_per_axis,
                                   num_classes=self.params.number_of_classes,
                                   class_to_color_mapping=self.params.class_to_color_mapping)



In [3]:
pipeline = MySimpleCNNInceptionModule(neighborhood_tiles=1,
                                      train_data_dir='data/Feit_colon-annotation-tiles-256-1-neighborhood/data_train/',
                                      valid_data_dir='data/Feit_colon-annotation-tiles-256-1-neighborhood/data_valid/')

ValueError: AdditiveAttention layer accepts inputs list of length 2 or 3, namely [query, value] or [query, value, key]. Given length: 9

In [ ]:
pipeline.execute_pipeline(perform_validation=False, perform_test_segmentation=False)



In [ ]:
def extract_ensemble_classifier():

    inputs = [keras.Input(shape=(12,)) for _ in range(9)]
    merge_layer = keras.layers.Add()(inputs)
    output = keras.layers.Dense(units = 12, activation = 'softmax')(merge_layer)

    model = keras.Model(inputs=inputs, outputs=output)

    model.summary()

    pipeline.model.summary()


extract_ensemble_classifier()

In [ ]:
pipeline.save_pipeline()

In [ ]:
#eval_model(pipeline.model,
#           pipeline.get_data_loader_validation(),
#           print_confusion_matrix=True,
#           save_misclassified=True)

In [ ]:
#pipeline = FeitDataPipeline.load_pipeline(pipeline_name=name)

In [ ]:
path = Path('data/Kather_5000')
from util.data_manipulation_scripts import generate_image_annotation_pairs

tiffs, _ = generate_image_annotation_pairs(Path('data/Feit_colon-annotation'))
for tiff in tiffs:
    pipeline.perform_segmentation(tiff, step=15)